# Filter out anomalies
This notebook takes anomalies from the previous step and only keeps the most intense anomalies. The idea idea behind this is that if there are many anomalies across many buildings on the same day it is likely that the anomalies are cause by an external factor. This external factor could for example be holidays or wind.

In [1]:
import orchest
import numpy as np
from pyclarify import *
from utils import Anomaly

In [2]:
data = orchest.get_inputs()

anomalies = data['anomalies_from_et']['anomalies_from_ett']

anomaly_count_before = [len(a_list) for a_list in anomalies.values()]

In [3]:
max_daily_anomalies = 40
anomalies_by_date = {}

for building, anomaly_list in anomalies.items():
    for anomaly in anomaly_list:
        if anomaly.start not in anomalies_by_date:
            anomalies_by_date[anomaly.start] = []
        
        anomalies_by_date[anomaly.start].append(anomaly)

# remove least intense anomalies from inital dictionary where the anomunt of anomalies on same day exceed max
for anomaly_list in anomalies_by_date.values():
    if len(anomaly_list) > max_daily_anomalies:
        anomaly_list.sort(key=lambda x: x.intensity, reverse=True)
        for anomaly_to_remove in anomaly_list[max_daily_anomalies+1:]:
            anomalies[anomaly_to_remove.building].remove(anomaly_to_remove)

In [4]:
anomaly_count_after = [len(anomaly_list) for anomaly_list in anomalies.values()]
count_reduction = [before - after for before, after in zip(anomaly_count_before, anomaly_count_after)]

print(f'mean anomalies per building:')
print(f'before {np.mean(anomaly_count_before):.1f}')
print(f'after {np.mean(anomaly_count_after):.1f}')
print(f'reduction {np.mean(count_reduction):.1f}')

mean anomalies per building:
before 83.0
after 65.6
reduction 17.4


In [5]:
filtered_anomalies = {
    'filtered_anomalies': anomalies
}

In [6]:
print('outputting filtered anomalies...')
orchest.output(filtered_anomalies, name='filtered_anomalies')
print('success')

outputting filtered anomalies...
success
